In [16]:
import sys
sys.path.append(".")
import importlib
# import datasets.lip_to_word_dataset 
import util.cv_utils
# importlib.reload(datasets.lip_to_word_dataset)
importlib.reload(util.cv_utils)
# from datasets.lip_to_word_dataset import LipReadingWordDataset
from util.cv_utils import collate_fn

import torch
from torch.utils.data import Dataset
from pathlib import Path
from torch.utils.data import DataLoader


In [17]:
from torch.utils.data import Dataset


class LipReadingWordDataset(Dataset):
    # Implement abstract funcs
    
    # constructor
    # pass in path object of proccessed folder
    def __init__(self, processed_dir):
        self.index_map = []
        self.cache = {} # data optimization as we save loading times for redudant multiple get item calls for data, etc: epochs
        processed_dir = Path(processed_dir)
        for folder in processed_dir.iterdir():
            folder = Path(folder)
            for data_file in folder.glob("*.pth"):            
                data = torch.load(data_file, map_location= 'cpu', weights_only=False)
                num_words = len(data["y_labels"])
                for i in range(num_words):
                    self.index_map.append((data_file, i)) 
                    # load in the indicies and path of each word token, and load actual data in real time for optimization

    def __len__(self):
        # Return total amount of word tokens
        return len(self.index_map)

    def __getitem__(self, idx):
        # Return part of sequence for each word using index map (index being which word in the file)
        data_path, index = self.index_map[idx]
        if data_path not in self.cache:
            self.cache[data_path] = torch.load(data_path, map_location= 'cpu', weights_only= False)
        # load from cache
        data = self.cache[data_path]
        
        # Load data into tensors at runtime
        x_feat, x_coords, x_veloc, y_labels = data["x_feat"][index], data["x_coords"][index], data["x_veloc"][index], data["y_labels"][index]
        return torch.tensor(x_feat, dtype= torch.float32), torch.tensor(x_coords, dtype= torch.float32), torch.tensor(x_veloc, dtype= torch.float32), torch.tensor(y_labels, dtype= torch.long)
        


In [18]:
# Load data
# folder = Path(r"C:\Projects\Lip_Reading\GRID\processed")
# Daniel's Path
folder = Path(r"C:\Users\User\OneDrive\Documents\Projects\Lip-Reading\GRID\processed")
dataset = LipReadingWordDataset(folder)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [19]:
# Data size for model
# processed_file = Path(r"C:\Projects\Lip_Reading\GRID\processed\s1\bbaf2n_data.pth") 
# Daniel's Path
processed_file = Path(r"C:\Users\User\OneDrive\Documents\Projects\Lip-Reading\GRID\processed\s1\bbaf2n_data.pth")
data = torch.load(processed_file, map_location='cpu', weights_only= False)
# Access the first word/sample's feature tensor shape
x_feat = data["x_feat"][0]  # first sample's engineered features
x_coords = data["x_coords"][0]  # first sample's coords
x_veloc = data["x_veloc"][0]  # first sample's velocities
y_label = data["y_labels"][0]

print(f"x_feat shape: {x_feat.shape}")   # should print something like (seq_len, feat_dim)
print(f"x_coords shape: {x_coords.shape}")
print(f"x_veloc shape: {x_veloc.shape}")
print(f"Label: {y_label}")


x_feat shape: (6, 8)
x_coords shape: (6, 18)
x_veloc shape: (6, 18)
Label: 4


In [20]:
# Define a simple model
import torch.nn as nn
import torch.nn.functional as F

class LipReadingModel(nn.Module):
    def __init__(self, x_feat, x_coords, x_veloc, hidden_size, num_classes):
        super(LipReadingModel, self).__init__()

        input_size = x_feat + x_coords + x_veloc
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x_dim, x_2d1, x_2d2):
        x = torch.cat([x_dim, x_2d1, x_2d2], dim=2) 
        _, (h_n, _) = self.lstm(x)  
        out = self.fc(h_n[-1])      
        return out

    def __getitem__(self, idx):
        x_dim = torch.tensor(self.X_dim[idx], dtype=torch.float32)
        x2d1  = torch.tensor(self.X_2d1[idx], dtype=torch.float32)
        x2d2  = torch.tensor(self.X_2d2[idx], dtype=torch.float32)
        label = torch.tensor(self.y[idx], dtype=torch.long)  # labels must be long for CrossEntropyLoss
        return x_dim, x2d1, x2d2, label

In [21]:
model = LipReadingModel(
    x_feat = 8,
    x_coords = 18,
    x_veloc = 18,
    hidden_size = 128,
    num_classes = 55
)

In [22]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# TODO Fix corruption in data ruining pad_sequence b/c length of zero, investigate
for epoch in range(10):
    model.train()
    total_loss = 0.0
    for x1, x2, x3, y in train_loader:

        # Move data to device
        x1 = x1.to(device)
        x2 = x2.to(device)
        x3 = x3.to(device)
        y = y.to(device)
        
        # Forward pass
        outputs = model(x1, x2, x3)
        loss = criterion(outputs, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        total_loss += loss.item()
        
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")


Epoch 1: Loss = 3.4034
Epoch 2: Loss = 4.6242
Epoch 3: Loss = 5.6361
Epoch 4: Loss = 2.0822
Epoch 5: Loss = 6.0423
Epoch 6: Loss = 4.9557
Epoch 7: Loss = 4.2192
Epoch 8: Loss = 4.2960
Epoch 9: Loss = 4.2306
Epoch 10: Loss = 3.2623


In [24]:
vocab = {
    "<pad>": 0,
    "<sos>": 1,
    "<eos>": 2,
    "sp": 3,
    "bin": 4,
    "lay": 5,
    "place": 6,
    "set": 7,
    "blue": 8,
    "green": 9,
    "red": 10,
    "white": 11,
    "at": 12,
    "by": 13,
    "in": 14,
    "with": 15,
    "zero": 16,
    "one": 17,
    "two": 18,
    "three": 19,
    "four": 20,
    "five": 21,
    "six": 22,
    "seven": 23,
    "eight": 24,
    "nine": 25,
    "again": 26,
    "now": 27,
    "please": 28,
    "soon": 29,
    "a": 30,
    "b": 31,
    "c": 32,
    "d": 33,
    "e": 34,
    "f": 35,
    "g": 36,
    "h": 37,
    "i": 38,
    "j": 39,
    "k": 40,
    "l": 41,
    "m": 42,
    "n": 43,
    "o": 44,
    "p": 45,
    "q": 46,
    "r": 47,
    "s": 48,
    "t": 49,
    "u": 50,
    "v": 51,
    "x": 52,
    "y": 53,
    "z": 54
}
inverse_vocab = {v: k for k, v in vocab.items()}

In [25]:
# Adding Metrics and Evaluation
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for x1, x2, x3, y in train_loader:
        x1 = x1.to(device)
        x2 = x2.to(device)
        x3 = x3.to(device)
        y = y.to(device)

        outputs = model(x1, x2, x3)
        predicted = outputs.argmax(1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

# Compute metrics
acc = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Final Training Accuracy: {acc:.2%}")

labels_list = list(range(len(vocab)))
target_names = [inverse_vocab[i] for i in labels_list]

# Optional detailed report
print(classification_report(all_labels, all_preds, labels=labels_list, target_names=target_names))

# Optional confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)


Final Training Accuracy: 44.27%
              precision    recall  f1-score   support

       <pad>       0.00      0.00      0.00         0
       <sos>       0.00      0.00      0.00         0
       <eos>       0.00      0.00      0.00         0
          sp       0.87      0.57      0.69       159
         bin       0.65      0.79      0.71       720
         lay       0.41      0.35      0.38       743
       place       0.65      0.70      0.67       765
         set       0.38      0.53      0.44       763
        blue       0.71      0.77      0.74       749
       green       0.41      0.41      0.41       749
         red       0.44      0.44      0.44       750
       white       0.53      0.72      0.61       750
          at       0.40      0.48      0.43       753
          by       0.47      0.42      0.44       746
          in       0.37      0.47      0.41       752
        with       0.42      0.74      0.53       744
        zero       0.46      0.28      0.35      

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average